# Views



## Principe
- Une vue permet d'enregistrer une requête complexe pour ne pas avoir à la ré-écrire à chaque fois.
- On peut ensuite utiliser une simple syntaxe SELECT

## Exemple de création

- Cette vue permet de calculer pour chaque produit :
    -  la quantité totale vendue
    -  le chiffre d'affaire corrigé des remises
    -  le prix moyen
    -  le nombre de commandes
    -  Et affiche le produit_id, le nom et la catégrie (c'est pour cela qu'ils sont tous dans le GROUP BY, juste pour l'affichage, pour le calcul, le produit_id suffirait

```sql
CREATE VIEW Summary AS
SELECT 
    p.product_id,
    p.product_name,
    c.category_name,
    s.company_name AS supplier_name,
    SUM(od.quantity) AS total_quantity_sold,
    SUM(od.unit_price * od.quantity * (1 - od.discount)) AS total_sales,
    AVG(od.unit_price) AS average_price,
    COUNT(DISTINCT o.order_id) AS total_orders
    
FROM order_details od
JOIN products p ON od.product_id = p.product_id
JOIN categories c ON p.category_id = c.category_id
JOIN suppliers s ON p.supplier_id = s.supplier_id
JOIN orders o ON od.order_id = o.order_id
GROUP BY p.product_id, p.product_name, c.category_name, s.company_name;
```

## Suppression 

- Pour supprimer cette vue :

```sql
DROP VIEW IF EXISTS Summary;
```

## Mise à jour des tables via mise à jour de la vue


- Pour qu'une vue puisse être modifiable (ie que la modifier modifie les tables correspondantes) :
    - la vue doit contenir une seule clause FROM
    - la vue ne doit pas contenir GROUP BY, HAVING, LIMIT, OFFSET, DISTINCT, WITH, UNION, INTERSECT, et EXCEPT
    - la selection retournée ne doit pas être calculée à partir d'une fonction (c'est en fait un peu plus compliqué)
 
## Exemple de mise à jour


```sql
-- Create vue
CREATE VIEW extract AS
SELECT *
FROM products

WHERE category_id = 2;


-- Insérer dans la vue
INSERT INTO extract 
(product_id, product_name, supplier_id, category_id, quantity_per_unit, unit_price, units_in_stock, units_on_order, reorder_level, discontinued) 
VALUES 
(78, 'Chai Tea', 1, 1, '10 boxes x 20 bags', 18.00, 39, 0, 10, 0),
(79, 'Coffee', 2, 2, '20 boxes x 30 bags', 23.50, 40, 0, 20, 0),
(80, 'Olive Oil', 3, 2, '12 - 500 ml bottles', 12.00, 80, 0, 30, 0);

-- Vérifier dans la table
SELECT * FROM products ORDER BY product_id DESC ;

-- Vérifier dans la vue
SELECT * FROM extract ORDER BY product_id DESC ;
```




**Attention : force la valeur de category_id à 2**


# Indexes

## Principe

- Les indexes permettent de faire des requêtes plus rapides, ils sont créés sur les colonnes que l'on utilise pour identifier les données
- Par défaut, un indexe est créé pour la PRIMARY KEY

## Exemple : avant l'indexe


```sql
CREATE TABLE address (
    address_id SERIAL PRIMARY KEY,
    city VARCHAR(50),
    email VARCHAR(255) UNIQUE
);

INSERT INTO address (city, email) VALUES ('Paris', 'hubert@paris.com');
INSERT INTO address (city, email) VALUES ('Lyon', 'jean@lyon.com');
INSERT INTO address (city, email) VALUES ('Nice', 'jack@nice.com');
INSERT INTO address (city, email) VALUES ('Marseille', 'kax@marseille.com');
INSERT INTO address (city, email) VALUES ('Toulouse', 'max@toulouse.com');
INSERT INTO address (city, email) VALUES ('Bordeaux', 'rené@bordeaux.com');
INSERT INTO address (city, email) VALUES ('Nantes', 'hugues@nantes.com');
INSERT INTO address (city, email) VALUES ('Strasbourg', 'marcel@strasbourg.com');
INSERT INTO address (city, email) VALUES ('Montpellier', 'mano@montpellier.com');
INSERT INTO address (city, email) VALUES ('Lille', 'yamamoto@lille.com');
INSERT INTO address (city, email) VALUES ('Rennes', 'manomato@rennes.com');
INSERT INTO address (city, email) VALUES ('Reims', 'mamamomototo@reims.com');

EXPLAIN SELECT * FROM address WHERE email = 'mano@montpellier.com';

```

## Exemple après l'indexe

```sql
CREATE INDEX idx_address_email ON address(email) ;
EXPLAIN SELECT * FROM address WHERE email = 'mano@montpellier.com' ;
```

## Créer un indexe sur plusieurs colonnes 

```sql
CREATE INDEX idx_address_city_email ON address(city, email);
```

**Attention, l'ordre est très important !!**

- PostgreSQL utilise l'index en commençant par la première colonne (ici city). Donc d'abord optimisé pour rechercher sur cette colonne
- L'index est plus efficace pour les requêtes qui filtrent ou trient d'abord sur city, puis sur email. Par exemple, une requête avec WHERE city = 'Paris' AND email = 'contact@paris.com', mais pas email = 'contact@paris.com' AND WHERE city = 'Paris'
- Si une requête filtre uniquement sur email sans mentionner city, l'index peut toujours être utilisé, mais beaucoup moins efficacement

## Supprimer un indexe
```sql
DROP INDEX idx_address_city_email;
```


# User Function

## Principe

- On peut créer des fonctions avec PostgreSQL pour encapsuler de la logique
- La syntaxe est  :

```sql
CREATE [OR REPLACE] FUNCTION function_name(param_list)
   RETURNS return_type 
   LANGUAGE plpgsql
  AS
$$
DECLARE 
-- variables
BEGIN
 -- logique
END;
$$
```
  

## Exemple

```sql
CREATE OR REPLACE FUNCTION total_freight_for_country(country_name VARCHAR)
RETURNS REAL AS $$
DECLARE
    total_freight REAL;
BEGIN
    SELECT SUM(freight) INTO total_freight
    FROM orders
    WHERE ship_country = country_name;

    RETURN total_freight;
END;
$$ LANGUAGE plpgsql;


-- Appel 
total_freight_for_country('USA')
-- Appel
SELECT total_freight_for_country('USA') AS total_freight_to_usa;

```

# Triggers

## Principe
- Les triggers permettent d'appeler une fonction automatiquement lors de certains évènements (INSERT, UPDATE, DELETE et TRUNCATE, qui sert à supprimer les données d'une table).
- Ils permettent par exemple de décharger une application en se réalisant des calculs, il est en revanche capital de savoir que le trigger existe (sinon on ne comprend rien) - il permet donc aussi de baisser la complexité d'une app
- Les triggers peuvent être déclenchés par ligne ou par évènement : si on insère 5 lignes, le premier cas le déclenche 5 fois, le second cas seulement une fois

## Un exemple par ligne (FOR EACH ROW)


- On écrit dans une table `answers` la réponse (un entier appelé number)  d'un utilisateur pour une question, qui est reliée à un exercice. On écrit également si la réponse est correcte.

- On souhaite automatique calculer le nombre de bonnes réponses pour un utilisateur et exercice dans une table `exercice_stat`, en le mettant à jour à chaque écriture dans `answers`.


```sql

CREATE DATABASE trigg;

\c trigg;
 
CREATE TABLE answers (
    id SERIAL PRIMARY KEY,
    user_id INTEGER,
    exercice_id INTEGER,
    number INTEGER,
    is_correct BOOLEAN
);

CREATE TABLE exercice_stats (
    id SERIAL PRIMARY KEY,
    exercice_id INTEGER,
    user_id INTEGER,
    cumulative_score INTEGER
);

---------------------------------
-- CREER LA FONCTION DU TRIGGER
---------------------------------

CREATE OR REPLACE FUNCTION update_exercice_stats()
RETURNS TRIGGER AS $$
BEGIN
    -- Vérifier si une entrée existe déjà pour cet exercice_id et user_id dans la table exercice_stats
    IF EXISTS (SELECT 1 FROM exercice_stats WHERE exercice_id = NEW.exercice_id AND user_id = NEW.user_id) THEN
        -- Mettre à jour la ligne existante en ajoutant 1 au score si la réponse est correcte
        IF NEW.is_correct THEN
            UPDATE exercice_stats SET cumulative_score = cumulative_score + 1
            WHERE exercice_id = NEW.exercice_id AND user_id = NEW.user_id;
        END IF;
    ELSE
        -- Créer une nouvelle ligne pour cet exercice_id et user_id avec cumulative_score initialisé à 1 ou 0
        INSERT INTO exercice_stats (exercice_id, user_id, cumulative_score) 
        VALUES (NEW.exercice_id, NEW.user_id, CASE WHEN NEW.is_correct THEN 1 ELSE 0 END);
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

---------------------------------
-- CREER LE TRIGGER
---------------------------------

CREATE TRIGGER after_answers_insert
AFTER INSERT ON answers
FOR EACH ROW
EXECUTE FUNCTION update_exercice_stats();

---------------------------------
-- INSERER DES REPONSES
---------------------------------


-- USER 1
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (1, 101, 35, TRUE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (1, 101, 20, TRUE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (1, 101, 20, TRUE);

INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (1, 102, 20, TRUE);

INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (1, 103, 30, TRUE);

-- USER 2 
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (2, 102, 40, FALSE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (2, 102, 15, FALSE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (2, 102, 10, TRUE);

INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (2, 103, 40, FALSE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (2, 103, 15, FALSE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (2, 103, 10, TRUE);

-- USER 3
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (3, 101, 25, FALSE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (3, 103, 5, FALSE);
INSERT INTO answers (user_id, exercice_id, number, is_correct) VALUES (3, 103, 5, FALSE);


---------------------------------
-- VERIFICATION
---------------------------------
SELECT * from exercice_stats;

```

## Supprimer un trigger 
```sql
DROP TRIGGER after_answers_insert;
-- ou 
DROP TRIGGER answers_after_insert ON answers;
```

## Désactiver un trigger 

```sql
ALTER TABLE answers
DISABLE TRIGGER answers_after_insert
```

## Désactiver tous les triggers


```sql
ALTER TABLE answers
DISABLE TRIGGER ALL
```